# COMP5331 Group 6 Project: Resilient k-Clustering

In [1]:
import numpy as np
import math
import copy
import random 

# Helper Function

In [98]:
def distance(point1, point2):
    # This function is to calculate the distance between 2 points
    distance_x_y = np.linalg.norm(np.array(point1) - np.array(point2))
    return distance_x_y

# Helper Class

In [99]:
# Modified from https://dev.to/theramoliya/python-kruskals-algorithm-for-minimum-spanning-trees-2bmb
class Graph:
    def __init__(self, vertices, graph=None):
        self.V = vertices
        if graph is None:
            self.graph = []
        else:
            self.graph = copy.deepcopy(graph)
        
    def copy(self):
        return Graph(self.V, self.graph)

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])
    
    def remove_edge(self, u, v, w):
        self.graph.remove([u, v, w])

    def kruskal_mst(self):
        def find(parent, i):
            if parent[i] == i:
                return i
            return find(parent, parent[i])

        def union(parent, rank, x, y):
            x_root = find(parent, x)
            y_root = find(parent, y)

            if rank[x_root] < rank[y_root]:
                parent[x_root] = y_root
            elif rank[x_root] > rank[y_root]:
                parent[y_root] = x_root
            else:
                parent[y_root] = x_root
                rank[x_root] += 1

        result = []
        i = 0
        e = 0

        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = [i for i in range(self.V)]
        rank = [0] * self.V

        while e < self.V - 1:
            u, v, w = self.graph[i]
            i += 1
            x = find(parent, u)
            y = find(parent, v)

            if x != y:
                e += 1
                result.append([u, v, w])
                union(parent, rank, x, y)

        return result


In [100]:
class Approx_algo:
    def __init__(self, dataset, k, seed=5331):
        self.dataset = dataset
        self.k = k
        self.seed = seed

    # Define KCluster
    class Cluster:
        def __init__(self):
            self.elements = [] # Initially, cluster has no points inside
            self.head = None 
            
    def clustering(self):
        def initialize_clusters(dataset, seed = None):
            # Since we need to initialize the first cluster, there must be someone who does that first
            cluster = Approx_algo.Cluster() # call class Cluster
            cluster.elements = dataset.tolist() # All data now become the point of cluster 1
            if seed is not None:
                random.seed(seed)
            cluster.head = random.choice(cluster.elements)
            return [cluster]
        
        def expand_clusters(clusters, j):
            # At this function, we will perform expansion

            # We will find the point with maximal distance to the head
            max_distance = -1
            v_i = None 

            for i in range(j - 1):
                current_cluster = clusters[i]

                for point in current_cluster.elements:
                    dist = distance(point, current_cluster.head)
                    if dist > max_distance:
                        max_distance = dist
                        v_i = point
            
            # Create new cluster B_(j + 1)
            new_cluster = Approx_algo.Cluster()
            new_cluster.head = v_i 
            new_cluster.elements = []
            
            # Move elements to the new cluster
            for i in range(j - 1):
                current_cluster = clusters[i]
                # print("head ", i+1, current_cluster.head)
                for point in current_cluster.elements:
                    # print("distance v_1", distance(point, v_i))
                    # print("distance head", distance(point, current_cluster.head))
                    if distance(point, v_i) <= distance(point, current_cluster.head):
                        # print("point ", point)
                        new_cluster.elements.append(point)
                
                # Delete the elements that was appended to new cluster
                current_cluster.elements = [element for element in current_cluster.elements if element not in new_cluster.elements]
                # print("Cluster ", i + 1, " : ", current_cluster.elements)

            # Add this new cluster to a list of cluster
            clusters.append(new_cluster)

            return clusters
    
        def get_heads(clusters):
            # Give me the list of current clusters head
            heads = []
            for cluster in clusters:
                heads.append(cluster.head)
            
            return heads

        clusters = initialize_clusters(self.dataset, self.seed)
        for self.k in range(2, self.k + 1): # note that it should be range(2, k+1), we start from 2 because we already initialize a cluster
            clusters = expand_clusters(clusters, self.k)

        # Get the heads of the clusters
        heads = get_heads(clusters)

        return heads


In [101]:
class resilient_k_center():
    def __init__(self, dataset, k, epsilon, lamb=0.1, seed=5331):
        self.dataset = dataset
        self.k = k
        self.epsilon = epsilon
        self.lamb = lamb
        self.number_of_centers = int(2 * self.k * np.log(1 / self.epsilon))
        self.seed = seed

    def resilient_k_center(self):
        # randomly assign centers (line 1)
        centers = self.dataset[np.random.choice(self.dataset.shape[0],
                                                self.number_of_centers,
                                                replace=False)]
        print("Centers: \n:", centers)

        # construct edges and weights (line 2-4)
        E = []
        w = {}
        for index_p, p in enumerate(self.dataset):
            for index_q, q in enumerate(self.dataset):
                if index_p < index_q:
                    if (p.tolist() in centers.tolist()) or (q.tolist() in centers.tolist()):
                        E.append((index_p, index_q))

                        # Integrated line 1-7 of algorithm Resilient-MST in the weight update below
                        if (p.tolist() in centers.tolist()) and (q.tolist() in centers.tolist()):
                            w[(index_p, index_q)] = 0
                        else:
                            alpha = np.random.rand()
                            i = math.ceil(alpha + math.log(distance(p, q), self.lamb))
                            w[(index_p, index_q)] = self.lamb ** (i - alpha)
        print(E)
        print(w)
        
        # construct weighted graph (line 5)
        g = Graph(len(self.dataset))
        for edge in E:
            index_p, index_q = edge
            g.add_edge(index_p, index_q, w[(index_p, index_q)])
            g.add_edge(index_q, index_p, w[(index_p, index_q)])
        print("weighted graph: \n", g.graph)    

        # construct MST (line 6)
        T = g.kruskal_mst()
        print("resilient MST: \n", T)

        # assign clusters (line 7-8)
        cluster = []
        for index_p, p in enumerate(self.dataset):
            if p.tolist() in centers.tolist():
                cluster.append((p, p))
            else:
                for edge in T:
                    index_u, index_v, _ = edge
                    if (index_p == index_u) and (self.dataset[index_v].tolist() in centers.tolist()):
                        cluster.append((p, self.dataset[index_v]))
                    elif (index_p == index_v) and (self.dataset[index_u].tolist() in centers.tolist()):
                        cluster.append((p, self.dataset[index_u]))
        
        print("Initial Cluster: \n", cluster)

        # find non-center vertices which incident to the heaviest edges (line 9)
        n = len(self.dataset)
        heaviest_edges = sorted(T, key=lambda item: item[2], reverse=True)[:int(self.epsilon * n)]
        print("Heaviest Edges: \n", heaviest_edges)
        L = set()   
        for edge in heaviest_edges:
            index_p, index_q, _ = edge
            if (self.dataset[index_p].tolist() not in centers.tolist()):
                L.add(index_p)
            if (self.dataset[index_q].tolist() not in centers.tolist()):
                L.add(index_q)
        print("L: \n", L)

        # centres selected by Algorithm Approx [27] (line 10)
        approx_algo = Approx_algo(self.dataset, self.k, self.seed)
        centers_approx = approx_algo.clustering()
        print("Centers selected by Approx: \n", centers_approx)

        # Update the center as the closest center of C' to each point in L (line 11)
        for index_p in L:
            min_dist = float('inf')
            closest_center = None
            for c in centers_approx:
                dist = distance(self.dataset[index_p], c)
                if dist < min_dist:
                    min_dist = dist
                    closest_center = c
            if (closest_center is not None):
                for index_c, cluster_pair in enumerate(cluster):
                    p, _ = cluster_pair
                    if np.array_equal(p, self.dataset[index_p]):
                        cluster[index_c] = (p, closest_center)
                        break
                    
        print("Final Cluster: \n", cluster)
        return cluster
        

# Test

In [102]:
test = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
k = 1
epilson = 0.3
model = resilient_k_center(test, k, epilson)
cluster = model.resilient_k_center()

Centers: 
: [[ 7  8]
 [ 9 10]]
[(0, 3), (0, 4), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
{(0, 3): 8.394079400821992, (0, 4): 1.134207058459542, (1, 3): 3.002091871985522, (1, 4): 2.165705309197452, (2, 3): 1.0695333251871488, (2, 4): 0.655603679481183, (3, 4): 0}
weighted graph: 
 [[0, 3, 8.394079400821992], [3, 0, 8.394079400821992], [0, 4, 1.134207058459542], [4, 0, 1.134207058459542], [1, 3, 3.002091871985522], [3, 1, 3.002091871985522], [1, 4, 2.165705309197452], [4, 1, 2.165705309197452], [2, 3, 1.0695333251871488], [3, 2, 1.0695333251871488], [2, 4, 0.655603679481183], [4, 2, 0.655603679481183], [3, 4, 0], [4, 3, 0]]
resilient MST: 
 [[3, 4, 0], [2, 4, 0.655603679481183], [0, 4, 1.134207058459542], [1, 4, 2.165705309197452]]
Initial Cluster: 
 [(array([1, 2]), array([ 9, 10])), (array([3, 4]), array([ 9, 10])), (array([5, 6]), array([ 9, 10])), (array([7, 8]), array([7, 8])), (array([ 9, 10]), array([ 9, 10]))]
Heaviest Edges: 
 [[1, 4, 2.165705309197452]]
L: 
 {1}
Centers selecte

In [103]:
test2 = np.array([[0, 8], [4, 8], [0, 4], [12, 4], [12, 0], [8, 0]])
k2 = 1
epilson2 = 0.3
model2 = resilient_k_center(test2, k2, epilson2)
cluster2 = model2.resilient_k_center()

Centers: 
: [[ 0  4]
 [12  4]]
[(0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5)]
{(0, 2): 0.41951288296193295, (0, 3): 2.670972829764518, (1, 2): 0.8491531708434129, (1, 3): 5.281557868857898, (2, 3): 0, (2, 4): 4.320047157428978, (2, 5): 3.1828420196047422, (3, 4): 2.9926182372809524, (3, 5): 4.1379806426038925}
weighted graph: 
 [[0, 2, 0.41951288296193295], [2, 0, 0.41951288296193295], [0, 3, 2.670972829764518], [3, 0, 2.670972829764518], [1, 2, 0.8491531708434129], [2, 1, 0.8491531708434129], [1, 3, 5.281557868857898], [3, 1, 5.281557868857898], [2, 3, 0], [3, 2, 0], [2, 4, 4.320047157428978], [4, 2, 4.320047157428978], [2, 5, 3.1828420196047422], [5, 2, 3.1828420196047422], [3, 4, 2.9926182372809524], [4, 3, 2.9926182372809524], [3, 5, 4.1379806426038925], [5, 3, 4.1379806426038925]]
resilient MST: 
 [[2, 3, 0], [0, 2, 0.41951288296193295], [1, 2, 0.8491531708434129], [3, 4, 2.9926182372809524], [2, 5, 3.1828420196047422]]
Initial Cluster: 
 [(array([0, 8]),

## Evalutaion

Gonzalez implementation from https://github.com/TSunny007/Clustering/blob/master/notebooks/Gonzalez.ipynb

In [ ]:
# Modified from https://github.com/TSunny007/Clustering/blob/master/notebooks/Gonzalez.ipynb
from scipy.spatial import distance
def max_dist(data, clusters):
    distances = np.zeros(len(data)) # we will keep a cumulative distance measure for all points
    for cluster_id, cluster in enumerate(clusters):
        for point_id, point in enumerate(data):
            if distance.euclidean(point,cluster) == 0.0:
                distances[point_id] = -math.inf # this point is already a cluster 
            if not math.isinf(distances[point_id]):
                distances[point_id] = distances[point_id] + distance.euclidean(point,cluster) 
    return data[np.argmax(distances)]

In [ ]:
def norm_dist(data, clusters):
    distances = np.zeros(len(data)) # we will keep a cumulative distance measure for all points
    for point_id, point in enumerate(data):
        for cluster_id, cluster in enumerate(clusters):
            if distance.euclidean(point,cluster) == 0.0:
                distances[point_id] = -math.inf # this point is already a cluster (obselete)
            if not math.isinf(distances[point_id]):
                # if a point is not obselete, then we add the distance to its specific bin
                distances[point_id] = distances[point_id] + math.pow(distance.euclidean(point,cluster),2) 
                # return the point which is furthest away from all the other clusters
    for distance_id, current_distance in enumerate(distances):
        if not math.isinf(current_distance): 
            distances[distance_id] = math.sqrt(current_distance/len(data))
    return data[np.argmax(distances)]

In [ ]:
def gonzalez(data, cluster_num, method = 'max'):
    clusters = []
    clusters.append(data[0]) # assign the first point to the first cluster
    while len(clusters) < cluster_num:
        if method is 'max':
            clusters.append(max_dist(data, clusters)) 
        if method is 'norm':
            clusters.append(norm_dist(data, clusters)) 
        # we add the furthest point from ALL current clusters
    return (clusters)

Carving algorithm implementation

In [ ]:
import random
def distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def carve(points, R, k):
    centers = []
    uncovered_indices = set(range(len(points)))  # Indices of uncovered points

    while uncovered_indices and len(centers) < k:
        # Randomly select an uncovered point
        idx = random.choice(list(uncovered_indices))
        center = points[idx]
        centers.append(center)

        # Mark all points within distance R from the new center as covered
        to_remove = []
        for i in uncovered_indices:
            if distance(center, points[i]) <= R:
                to_remove.append(i)

        # Remove covered points from uncovered set
        uncovered_indices.difference_update(to_remove)

    return centers

In [ ]:

def find_minimum_R(points, k, R_start, R_end, step=0.1):
    best_R = None

    R = R_start
    while R <= R_end:
        centers = carve(points, R, k)
        if len(centers) <= k:  # Check if we opened at most k centers
            best_R = R  # Update best R found
            R -= step  # Try a smaller R
        else:
            R += step  # Increase R
    return best_R

## K-Medoid 

In [ ]:
import numpy as np

def distance_matrix(points):
    return np.linalg.norm(points[:, np.newaxis] - points, axis=2)

def assign_clusters(points, medoids):
    distances = distance_matrix(points)
    cluster_assignment = np.argmin(distances[:, medoids], axis=1)
    return cluster_assignment

def update_medoids(points, clusters, k):
    new_medoids = []
    for i in range(k):
        # Get points in the current cluster
        cluster_points = points[clusters == i]
        if len(cluster_points) == 0:
            continue

        # Calculate the cost for each point in the cluster as a potential medoid
        costs = []
        for point in cluster_points:
            cost = np.sum(np.linalg.norm(cluster_points - point, axis=1))
            costs.append(cost)

        # Find the point with the minimum cost
        new_medoid = cluster_points[np.argmin(costs)]
        new_medoids.append(np.where((points == new_medoid).all(axis=1))[0][0])

    return np.array(new_medoids)

def pam(points, k, max_iter=100):
    # Randomly select k initial medoids
    initial_indices = np.random.choice(len(points), k, replace=False)
    medoids = initial_indices.copy()

    for _ in range(max_iter):
        # Step 1: Assign clusters based on current medoids
        clusters = assign_clusters(points, medoids)

        # Step 2: Update medoids
        new_medoids = update_medoids(points, clusters, k)

        # Check for convergence
        if np.array_equal(medoids, new_medoids):
            break

        medoids = new_medoids

    return medoids, clusters

## Metrics

In [1]:
def fraction_points_changing_cluster(old_clusters, new_clusters):
    changes = np.sum(old_clusters != new_clusters)
    total_points = len(old_clusters)
    return changes / total_points


In [2]:
def solution_cost(points, clusters, medoids):
    max_distance = 0
    for i, point in enumerate(points):
        medoid = medoids[clusters[i]]
        distance = np.linalg.norm(point - points[medoid])
        max_distance = max(max_distance, distance)
    return max_distance

In [3]:
def number_of_clusters(clusters):
    """Count the number of unique clusters formed."""
    return len(np.unique(clusters))